In [1]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [2]:
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

final_entities=load_df_from_csv_with_embedding(f"/Users/boyuren/Documents/multi_head_graph_rag/backup_data/multihop_rag/final_entities_with_embeddings.csv",['entity_embedding'])
final_entities

,entity_name,entity_type,description,id,entity_embedding
0,49ers,ORGANIZATION,49ers is a football team,1,"[0.04144171625375748, -0.007414397317916155, -..."
1,Barbarians,CONCEPT,a character class in the game,2,"[0.0122455395758152, -0.007463064510375261, -0..."
2,Bills,ORGANIZATION,The team that Allen plays for,3,"[0.030726933851838112, 0.0030639127362519503, ..."
3,Bolt Mobility,COMPANY,Bolt Mobility is introducing distance-based pr...,4,"[0.004269156605005264, -0.011844240128993988, ..."
4,Crimson Tide,TEAM,Crimson Tide is the nickname of the Alabama fo...,5,"[0.004862545058131218, -0.003534935647621751, ..."
...,...,...,...,...,...
23565,家庭自动化,CONCEPT,家庭自动化是指通过网络连接和控制家庭设备的技术。,23569,"[0.02464906871318817, 0.004047214984893799, -0..."
23566,昂贵的,CONCEPT,昂贵的是形容 soundbar 的形容词,23570,"[0.04650348424911499, 0.02652094140648842, -0...."
23567,智能扬声器,CONCEPT,智能扬声器是一种可以通过网络连接和控制的扬声器。,23571,"[0.03134648874402046, 0.013866862282156944, -0..."
23568,杜比 atmos,CONCEPT,杜比 atmos is a surround sound technology.,23572,"[0.01925036683678627, 0.018349502235651016, -0..."


In [3]:
final_relationships=load_df_from_csv_with_embedding(f"/Users/boyuren/Documents/multi_head_graph_rag/backup_data/multihop_rag/final_relationships_with_embeddings.csv",['relationship_embedding'])
final_relationships

,source_entity,target_entity,relationship_type,relationship_description,source_id,target_id,relationship_embedding
0,Li,Falun Gong,Affiliated-with,Li is affiliated with Falun Gong.,5841,3963,"[0.006164404563605785, 0.004094846546649933, -..."
1,Epoch Times,Li,Worked-for,Epoch Times worked for Li.,3835,5841,"[0.014970953576266766, -0.01761895790696144, 0..."
2,The Epoch Times,Location,Located-in,The Epoch Times is located in multiple countries.,9282,5932,"[0.022465074434876442, -0.007946091704070568, ..."
3,Microsoft,The Epoch Times,Affiliated-with,Microsoft is affiliated with The Epoch Times.,6510,9282,"[0.024022087454795837, -0.024629345163702965, ..."
4,California,The Epoch Times,Located-in,California is located in The Epoch Times.,2502,9282,"[0.02883162721991539, -0.014359557069838047, -..."
...,...,...,...,...,...,...,...
26033,Lille,Alphonso Davies,Part-of,Lille is a part of Davies,5873,1515,"[0.009421243332326412, 0.025672562420368195, -..."
26034,Canada Soccer,"November 18, 2023",Happened-at,"November 18, 2023 is an event that happened at...",2535,7125,"[0.05737772211432457, 0.003261846723034978, -0..."
26035,David,Lille,Part-of,David is a part of Lille,3263,5873,"[-0.006869114935398102, 0.004632634576410055, ..."
26036,David,Milan Borjan,Part-of,David is a part of Borjan,3263,6561,"[0.024512803182005882, -0.0016717436956241727,..."


In [6]:
import torch
import pandas as pd
from sklearn.metrics import adjusted_mutual_info_score
from src.gnn_clustering.train import train_model_multi_head
from src.gnn_clustering.evaluate import (
    get_embeddings,
    get_embeddings_list,
    leiden_embeddings_clustering,
    leiden_clustering
)
from src.gnn_clustering.data_loader import load_random_data
from src.gnn_clustering.model import get_multi_head_model, get_model
from src.gnn_clustering.utils import get_device, get_dense_adj


# 设置设备
device = get_device()

# 计算多个头之间的互信息
def calculate_ami_between_heads(cluster_labels):
    num_heads = len(cluster_labels)
    ami_matrix = [[0] * num_heads for _ in range(num_heads)]
    
    # 计算每两个头之间的 AMI
    for i in range(num_heads):
        for j in range(i + 1, num_heads):
            ami = adjusted_mutual_info_score(cluster_labels[i], cluster_labels[j])
            ami_matrix[i][j] = ami
            ami_matrix[j][i] = ami

    return ami_matrix

# 将社区划分转换为节点标签
def communities_to_labels(communities, num_nodes):
    labels = [-1] * num_nodes
    for community_id, community in enumerate(communities):
        for node in community:
            labels[node] = community_id
    return labels

# 测试模型性能的方法
def test_model_performance(num_nodes, num_edges):
    results = []

    # 加载随机数据
    random_data = load_random_data(num_nodes, num_edges)
    random_data = random_data.to(device)

    # 使用Leiden算法进行聚类，获取簇数和模块度
    initial_communities_leiden, initial_modularity_leiden = leiden_clustering(random_data)
    initial_num_clusters_leiden = len(initial_communities_leiden)

    # 初始模型和优化器
    num_heads = 3
    multi_head_model = get_multi_head_model(data=random_data, device=device, num_heads=num_heads)
    single_head_model = get_model(random_data, device=device)
    optimizer = torch.optim.Adam(multi_head_model.parameters(), lr=0.01)

    # 获取密集邻接矩阵
    adj = get_dense_adj(random_data.edge_index, device=device)

    # 初始嵌入和KMeans聚类模块度
    initial_embeddings = get_embeddings(single_head_model, random_data, device=device)
    
    # 替换为封装好的方法
    leiden_communities_before_training, leiden_modularity_before_training = leiden_embeddings_clustering(initial_embeddings, K=3)

    # 模型训练
    trained_model = train_model_multi_head(multi_head_model, random_data, adj, optimizer, num_heads)
    embeddings_list_after_training = get_embeddings_list(trained_model, random_data, device)

    print(f'初始 Leiden 算法的模块度: {initial_modularity_leiden:.4f}')
    print(f'初始 Leiden 算法得到的簇数: {initial_num_clusters_leiden}')
    print(f'基于嵌入的 Leiden 算法的模块度: {leiden_modularity_before_training:.4f}')
    print(f'基于嵌入的 Leiden 算法得到的簇数: {len(leiden_communities_before_training)}')

    # 存储每个头的聚类标签
    clustering_results = []

    # 基于训练后嵌入再次调用封装好的Leiden聚类方法
    for post_training_embeddings in embeddings_list_after_training:
        leiden_communities_after_training, leiden_modularity_after_training = leiden_embeddings_clustering(post_training_embeddings, K=3)
        print(f'基于训练后嵌入的 Leiden 算法的模块度: {leiden_modularity_after_training:.4f}')
        print(f'基于训练后嵌入的 Leiden 算法得到的簇数: {len(leiden_communities_after_training)}')

        # 将社区划分转换为节点标签
        predicted_labels = communities_to_labels(leiden_communities_after_training, random_data.num_nodes)
        clustering_results.append(predicted_labels)

    # 计算多个头之间的 AMI
    ami_matrix = calculate_ami_between_heads(clustering_results)

    # 输出 AMI 矩阵
    print("\n多个头之间的 Adjusted Mutual Information (AMI) 矩阵：")
    for row in ami_matrix:
        print(row)


# 执行测试
df_report = test_model_performance(num_nodes=10240, num_edges=15000)


Epoch 1, Loss: 70479.3359
Epoch 20, Loss: 1033.6438
Epoch 40, Loss: 224.2019
Epoch 60, Loss: 56.3746
Epoch 80, Loss: 39.3870
Epoch 100, Loss: 35.1710
Epoch 120, Loss: 33.3453
Epoch 140, Loss: 32.2989
Epoch 160, Loss: 31.6075
Epoch 180, Loss: 31.1032
Epoch 200, Loss: 30.7363
初始 Leiden 算法的模块度: 0.6672
初始 Leiden 算法得到的簇数: 666
基于嵌入的 Leiden 算法的模块度: 0.8368
基于嵌入的 Leiden 算法得到的簇数: 77
基于训练后嵌入的 Leiden 算法的模块度: 0.9295
基于训练后嵌入的 Leiden 算法得到的簇数: 75
基于训练后嵌入的 Leiden 算法的模块度: 0.9264
基于训练后嵌入的 Leiden 算法得到的簇数: 62
基于训练后嵌入的 Leiden 算法的模块度: 0.9298
基于训练后嵌入的 Leiden 算法得到的簇数: 84

多个头之间的 Adjusted Mutual Information (AMI) 矩阵：
[0, 0.06215610428384004, 0.0671720506215931]
[0.06215610428384004, 0, 0.0659784378563705]
[0.0671720506215931, 0.0659784378563705, 0]
